In [ ]:
from typing_extensions import TypedDict
from typing import List

from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model

from pydantic import BaseModel

llm = init_chat_model("openai:gpt-4o")

In [ ]:
class State(TypedDict):
    dish: str
    ingredients: list[dict]
    recipe_steps: str
    plating_instructions: str


class Ingredient(BaseModel):
    name: str
    quantity: str
    unit: str


class IngredientsOutput(BaseModel):
    ingredients: List[Ingredient]

In [ ]:
def list_ingredients(state: State):
    structured_llm = llm.with_structured_output(IngredientsOutput)
    ing_res = structured_llm.invoke(
        f"""
        {state["dish"]}를 만드는데 필요한 재료들을 5~8개 사이로 정리해서 알려줘.
        """
    )
    return {"ingredients": ing_res.ingredients}


def create_recipe(state: State):
    print("create_recipe 🔥")
    recipe_res = llm.invoke(
        f"""
        {state["dish"]}를 만들기위한 재료 목록이야 {state["ingredients"]}
        이재료들을 조리 순서에 맞춰서 레시피를 완성해줘.
        10단계 이상 넘지않고 각각 200단어 정도의 길이로 만들어줘.
        최종 요리가 어떤 모습일지 100단어 이내로 설명하는 부분을 넣어줘.
        """
    )
    return {"recipe_steps": recipe_res.content}


def discribe_plating(state: State):
    plating_res = llm.invoke(
        f"""
        {state["dish"]}를 만든 과정은 {state["recipe_steps"]} 이런데
        최종 접시에 플레이팅 하면 좋은 전략을 3가지 제안해줘.
        """
    )

    return {"plating_instructions": plating_res.content}

def gate (state: State):
    ingredients = state["ingredients"]
    print("replay 🔥", len(ingredients))
    if len(ingredients) > 7 or len(ingredients) < 5:
        return False
    return True


In [ ]:
graph_builder = StateGraph(State)

(
    graph_builder
    .add_node("list_ingredients", list_ingredients)
    .add_node("create_recipe", create_recipe)
    .add_node("discribe_plating", discribe_plating)

    .add_edge(START, "list_ingredients")
    .add_conditional_edges("list_ingredients", gate, {
        True: "create_recipe",
        False: "list_ingredients",
    })
    .add_edge("create_recipe", "discribe_plating")
    .add_edge("discribe_plating", END)
)

graph = graph_builder.compile()

graph.invoke(
    {
        "dish": "피자"
    }
)

In [ ]:
display(graph)